# 🚀 Nasdaq Trader - Production Pipeline

**Professional Trading Analysis from YouTube Videos**

## 🎯 Overview
This production-ready notebook provides a clean, professional interface for analyzing Turkish trading videos and generating actionable Nasdaq trading reports.

## 📋 Prerequisites
- Python 3.8+ environment
- Required dependencies installed (`pip install -r requirements.txt`)
- `GEMINI_API_KEY` environment variable set
- Video URLs in `video_list.txt`

## 🚀 Quick Start
1. **Setup**: Run the setup cell below
2. **Configure**: Add video URLs to `video_list.txt`
3. **Execute**: Run the processing pipeline
4. **Review**: Check generated reports in `summary/` folder


In [1]:
# Production Setup and Configuration
import os
import sys
import warnings
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Add current directory to path
sys.path.append(os.getcwd())

# Check environment
print("🔧 Production Environment Setup")
print("=" * 50)

# Check Python version
print(f"Python version: {sys.version}")

# Check required environment variables
gemini_key = os.getenv('GEMINI_API_KEY')
if gemini_key:
    print("✅ GEMINI_API_KEY found")
else:
    print("❌ GEMINI_API_KEY not found - please set this environment variable")

# Check video list
if os.path.exists('video_list.txt'):
    with open('video_list.txt', 'r', encoding='utf-8') as f:
        video_urls = [line.strip() for line in f if line.strip() and not line.startswith('#')]
    print(f"✅ Found {len(video_urls)} videos in video_list.txt")
else:
    print("⚠️  video_list.txt not found - please create it with video URLs")

print("\n🚀 Environment ready for production processing!")


🔧 Production Environment Setup
Python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
✅ GEMINI_API_KEY found
✅ Found 1 videos in video_list.txt

🚀 Environment ready for production processing!


## 🎬 Video Processing Pipeline

**Execute the complete trading analysis pipeline**


In [ ]:
# Self-Contained Trading Analysis Engine
import os
import sys
import time
import multiprocessing
import concurrent.futures
import logging
from pathlib import Path
import psutil
import yaml
from datetime import datetime
import whisper
import google.generativeai as genai
from yt_dlp import YoutubeDL
import yfinance as yf
from dotenv import load_dotenv
import warnings
import json

# Load environment variables
load_dotenv()
warnings.filterwarnings("ignore")

class SelfContainedNasdaqTrader:
    def __init__(self):
        self.config = self.load_config()
        self.setup_logging()
        self.system_info = self.get_system_info()
        self.optimal_settings = self.calculate_optimal_settings()
        
        print(f"🚀 Self-Contained Nasdaq Trader Initialized")
        print(f"   System: {self.system_info['cpu_cores']} cores, {self.system_info['ram_gb']:.1f}GB RAM")
        print(f"   Optimal: {self.optimal_settings['parallel_videos']} parallel videos")
    
    def load_config(self):
        """Load configuration with defaults"""
        try:
            with open('config.yaml', 'r', encoding='utf-8') as f:
                return yaml.safe_load(f)
        except:
            return {
                "ACCELERATION": {
                    "parallel_videos": 2,
                    "max_workers": 4,
                    "use_gpu": False,
                    "optimize_memory": True
                },
                "MODELS": {
                    "whisper_model": "small",
                    "gemini_model": "gemini-2.5-flash"
                }
            }
    
    def get_system_info(self):
        """Get system information for optimization"""
        return {
            "cpu_cores": multiprocessing.cpu_count(),
            "ram_gb": psutil.virtual_memory().total / (1024**3),
            "available_ram_gb": psutil.virtual_memory().available / (1024**3),
            "cpu_usage": psutil.cpu_percent(interval=1)
        }
    
    def calculate_optimal_settings(self):
        """Calculate optimal processing settings based on system specs"""
        cpu_cores = self.system_info['cpu_cores']
        available_ram = self.system_info['available_ram_gb']
        
        if cpu_cores >= 8:
            parallel_videos = min(4, cpu_cores // 2)
            max_workers = cpu_cores
        elif cpu_cores >= 4:
            parallel_videos = min(3, cpu_cores // 2)
            max_workers = cpu_cores
        else:
            parallel_videos = 1
            max_workers = max(2, cpu_cores)
        
        if available_ram >= 16:
            batch_size = 4
            quality_mode = "balanced"
        elif available_ram >= 8:
            batch_size = 3
            quality_mode = "fast"
        else:
            batch_size = 2
            quality_mode = "fast"
        
        return {
            "parallel_videos": parallel_videos,
            "max_workers": max_workers,
            "batch_size": batch_size,
            "quality_mode": quality_mode,
            "use_gpu": self.check_gpu_availability()
        }
    
    def check_gpu_availability(self):
        """Check if GPU is available for processing"""
        try:
            import torch
            return torch.cuda.is_available()
        except ImportError:
            return False
    
    def setup_logging(self):
        """Setup logging configuration"""
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s | %(levelname)s | %(message)s",
            handlers=[
                logging.StreamHandler(),
                logging.FileHandler("logs/self_contained_trader.log")
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    def optimize_system(self):
        """Optimize system for better performance"""
        try:
            p = psutil.Process()
            p.nice(psutil.HIGH_PRIORITY_CLASS)
            self.logger.info("System optimized for high performance")
        except Exception as e:
            self.logger.warning(f"Could not optimize system: {e}")
    
    def load_video_urls(self):
        """Load video URLs from various sources"""
        urls = []
        
        # Try environment variables first
        env_url = os.getenv('VIDEO_URL')
        env_urls = os.getenv('VIDEO_URLS')
        
        if env_url:
            urls.append(env_url)
        elif env_urls:
            urls.extend([url.strip() for url in env_urls.split(',') if url.strip()])
        
        # Fall back to video_list.txt
        if not urls and os.path.exists('video_list.txt'):
            try:
                with open('video_list.txt', 'r', encoding='utf-8') as f:
                    urls = [line.strip() for line in f if line.strip() and not line.startswith('#')]
            except Exception as e:
                self.logger.error(f"Error reading video_list.txt: {e}")
        
        return urls
    
    def download_video(self, url):
        """Download video and extract audio with proper caching"""
        try:
            os.makedirs('video_cache', exist_ok=True)
            
            # Extract video ID from URL first
            video_id = self.extract_video_id(url)
            if not video_id:
                raise Exception("Could not extract video ID from URL")
            
            # Check for existing audio files (any date)
            import glob
            existing_files = []
            for ext in ['m4a', 'wav', 'mp3', 'webm']:
                pattern = f'video_cache/{video_id}_*.{ext}'
                existing_files.extend(glob.glob(pattern))
            
            if existing_files:
                # Use the most recent existing file
                existing_file = max(existing_files, key=os.path.getctime)
                self.logger.info(f"Using cached audio: {existing_file}")
                return existing_file
            
            # Only download if no cached file exists
            self.logger.info(f"Downloading new video: {video_id}")
            
            # Get current date for new cache filename
            date_str = datetime.now().strftime('%Y%m%d')
            
            ydl_opts = {
                'format': 'bestaudio[ext=m4a]/bestaudio/best',
                'outtmpl': f'video_cache/%(id)s_{date_str}.%(ext)s',
                'noplaylist': True,
                'quiet': True,
                'no_warnings': True,
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'm4a',
                    'preferredquality': '192',
                }]
            }
            
            with YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(url, download=True)
                downloaded_video_id = info.get('id', 'unknown')
                
                for ext in ['m4a', 'wav', 'mp3', 'webm']:
                    audio_path = f'video_cache/{downloaded_video_id}_{date_str}.{ext}'
                    if os.path.exists(audio_path):
                        self.logger.info(f"Downloaded and cached: {audio_path}")
                        return audio_path
                
                raise Exception("Audio file not found after download")
                
        except Exception as e:
            self.logger.error(f"Download failed for {url}: {e}")
            return None
    
    def extract_video_id(self, url):
        """Extract video ID from YouTube URL"""
        try:
            import re
            # Handle various YouTube URL formats
            patterns = [
                r'(?:youtube\.com\/watch\?v=|youtu\.be\/|youtube\.com\/embed\/)([a-zA-Z0-9_-]{11})',
                r'youtube\.com\/watch\?.*v=([a-zA-Z0-9_-]{11})'
            ]
            
            for pattern in patterns:
                match = re.search(pattern, url)
                if match:
                    return match.group(1)
            return None
        except Exception as e:
            self.logger.error(f"Error extracting video ID: {e}")
            return None
    
    def transcribe_audio(self, audio_path):
        """Transcribe audio using Whisper with caching"""
        try:
            os.makedirs('transcript_cache', exist_ok=True)
            
            date_str = datetime.now().strftime('%Y%m%d')
            video_id = os.path.basename(audio_path).split('.')[0].split('_')[0]
            transcript_cache_path = f'transcript_cache/{video_id}_{date_str}.txt'
            
            if os.path.exists(transcript_cache_path):
                self.logger.info(f"Using cached transcript for {video_id}")
                with open(transcript_cache_path, 'r', encoding='utf-8') as f:
                    return f.read()
            
            self.logger.info(f"Transcribing audio: {audio_path}")
            model = whisper.load_model(self.config.get('MODELS', {}).get('whisper_model', 'small'))
            result = model.transcribe(audio_path, language='tr')
            transcript_text = result['text']
            
            with open(transcript_cache_path, 'w', encoding='utf-8') as f:
                f.write(transcript_text)
            self.logger.info(f"Transcript cached: {transcript_cache_path}")
            
            return transcript_text
        except Exception as e:
            self.logger.error(f"Transcription failed: {e}")
            return None
    
    def generate_analysis(self, transcript):
        """Generate AI analysis using Gemini"""
        try:
            api_key = os.getenv('GEMINI_API_KEY')
            if not api_key:
                raise Exception("GEMINI_API_KEY not found in environment")
            
            genai.configure(api_key=api_key)
            model = genai.GenerativeModel(self.config.get('MODELS', {}).get('gemini_model', 'gemini-2.5-flash'))
            
            prompt = f"""
            As an experienced Nasdaq portfolio manager, analyze this Turkish trading video transcript and create a professional trading report.
            
            TRANSCRIPT:
            {transcript}
            
            Create a comprehensive trading analysis report in this EXACT format:
            
            **IMPORTANT LANGUAGE REQUIREMENTS:**
            - Generate ALL content in Turkish for Turkish day/swing traders
            - Use Turkish trading terminology and expressions
            - Keep the report concise and action-oriented
            - Focus on practical trading information
            
            # TRADİNG ANALİZ RAPORU
            
            ## 📊 VİDEO BİLGİLERİ
            - **Tarih**: [Videoda belirtilen tarih, yoksa bugünün tarihi]
            - **Kanal**: [Kanal adı veya yayıncı]
            - **Video Başlığı**: [Video başlığı]
            - **Rapor Oluşturulma**: {datetime.now().strftime('%d %B %Y, %H:%M')}
            - **Not**: Bu rapor sadece video içeriğine dayanmaktadır, tahmin içermez
            
            ## 🎯 ÖZET
            [2-3 cümle ile ana trading fırsatları ve piyasa görünümü]
            
            ## 📈 TRADİNG FİRSATLARI
            [Her ticker için tek kapsamlı bölüm oluştur - tüm bilgileri bir arada]
            
            ### [TICKER] - [Şirket/Asset Adı]
            - **Videoda Bahsedilen**: [Hangi dakikada/saniyede bahsedildi - örnek: 5:23, 12:45]
            - **Fiyat**: [Mevcut fiyat, belirtilmişse - yoksa boş bırak]
            - **Öneri**: [AL/SAT/TUT] - [Gerekçe, belirtilmişse - yoksa boş bırak]
            - **Giriş**: [Fiyat, belirtilmişse - yoksa boş bırak]
            - **Hedef**: [Fiyat, belirtilmişse - yoksa boş bırak]
            - **Stop**: [Fiyat, belirtilmişse - yoksa boş bırak]
            - **Süre**: [Kısa/Orta/Uzun vadeli, belirtilmişse - yoksa boş bırak]
            - **Teknik Analiz**: [Destek/Direnç seviyeleri, grafik formasyonları, belirtilmişse - yoksa boş bırak]
            - **Piyasa Haberleri**: [Pozitif/negatif katalizörler, belirtilmişse - yoksa boş bırak]
            - **Risk Faktörleri**: [Riskler, belirtilmişse - yoksa boş bırak]
            - **Zamanlama**: [Hemen (0-24 saat) ve Kısa vadeli (1-7 gün) eylemler, belirtilmişse - yoksa boş bırak]
            
            [Her unique ticker/asset için bu bölümü tekrarla]
            
            ## 🚀 HIZLI KAZANÇLAR
            ### Hemen Alınacak Aksiyonlar
            - [Videoda bahsedilen acil trading aksiyonları]
            - [0-24 saat içinde yapılması gerekenler]
            - [Bu hafta için öncelikli eylemler]
            
            **CRITICAL ANTI-HALLUCINATION REQUIREMENTS:**
            
            🚫 **STRICT PROHIBITIONS:**
            - NEVER add tickers, prices, or information not explicitly mentioned in the transcript
            - NEVER use external knowledge or current market data
            - NEVER assume or infer information not directly stated
            - NEVER add technical analysis not explicitly described in the video
            - NEVER include market news or events not mentioned in the transcript
            
            ✅ **MANDATORY REQUIREMENTS:**
            1. ONLY include tickers and assets explicitly mentioned in the transcript
            2. ONLY include prices that are explicitly stated in the video
            3. ONLY include technical analysis that is explicitly described
            4. ONLY include trading ideas that are explicitly mentioned
            5. If information is not in the transcript, state "Not mentioned in video"
            6. Use exact quotes from the transcript when possible
            7. Clearly mark any assumptions or interpretations as "Based on transcript interpretation"
            8. Validate all ticker symbols (use standard format like AAPL, MSFT, etc.)
            9. If prices are mentioned, include them; if not, state "Price not specified in video"
            10. Be specific about entry/exit points only if explicitly mentioned
            11. Focus on actionable information that can be executed on NASDAQ
            12. Maintain professional trading report format
            
            🎯 **CRITICAL TICKER ORGANIZATION REQUIREMENTS:**
            13. Each ticker/asset must appear ONLY ONCE in the entire report
            14. Create ONE comprehensive section per ticker with ALL information about that ticker
            15. Include exact timestamps when tickers/assets are mentioned (e.g., "5:23", "12:45")
            16. Consolidate all information about each ticker into its dedicated section
            17. Do NOT repeat the same ticker in multiple sections
            18. Group all related information (prices, analysis, recommendations) under each ticker's section
            19. If a ticker is mentioned multiple times in the video, combine all information into ONE section
            20. Use the "Videoda Bahsedilen" field to show ALL timestamps where the ticker was mentioned
            
            🔍 **SOURCE VERIFICATION:**
            - Every piece of information must be traceable to the transcript
            - Use phrases like "According to the video" or "The speaker mentioned"
            - If uncertain, state "Unclear from transcript" rather than guessing
            - Never fill in gaps with external knowledge
            
            📝 **REPORTING STANDARDS:**
            - NEVER use predicted values, estimates, or future dates (e.g., "06 Haziran 2024, 15:30 (Tahmini)")
            - NEVER write "Videoda belirtilmemiş" or any placeholder text
            - NEVER generate fake dates - use the exact current date and time provided in the template
            - If no trading ideas are mentioned, leave the section completely blank
            - If no tickers are mentioned, leave the section completely blank
            - If no prices are mentioned, leave the price fields completely blank
            - If information is not mentioned, leave the field completely empty
            - Always prioritize accuracy over completeness
            - Only include information that is explicitly mentioned in the video
            - Include exact timestamps when tickers/assets are mentioned (e.g., "5:23", "12:45")
            - Use only current/past information from the video, no future predictions
            - CRITICAL: Use the exact date format provided in the template - do not change or modify it
            """
            
            response = model.generate_content(prompt)
            return response.text
            
        except Exception as e:
            self.logger.error(f"AI analysis failed: {e}")
            return None
    
    def save_report(self, analysis, url):
        """Save analysis report to file"""
        try:
            os.makedirs('summary', exist_ok=True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            video_id = url.split('v=')[-1].split('&')[0] if 'v=' in url else 'unknown'
            
            txt_filename = f'summary/report_{video_id}_{timestamp}.txt'
            with open(txt_filename, 'w', encoding='utf-8') as f:
                f.write(f"Trading Analysis Report\n")
                f.write(f"Video URL: {url}\n")
                f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"{'='*50}\n\n")
                f.write(analysis)
            
            json_filename = f'summary/report_{video_id}_{timestamp}.json'
            report_data = {
                'url': url,
                'timestamp': timestamp,
                'analysis': analysis,
                'generated_at': datetime.now().isoformat()
            }
            
            with open(json_filename, 'w', encoding='utf-8') as f:
                json.dump(report_data, f, indent=2, ensure_ascii=False)
            
            print(f"Report saved: {txt_filename}")
            
        except Exception as e:
            print(f"Failed to save report: {e}")
    
    def process_single_video(self, url):
        """Process a single video with all steps"""
        start_time = time.time()
        
        try:
            self.logger.info(f"Processing: {url}")
            
            # Download video
            audio_path = self.download_video(url)
            if not audio_path:
                raise Exception("Failed to download video")
            
            # Transcribe audio
            transcript = self.transcribe_audio(audio_path)
            if not transcript:
                raise Exception("Failed to transcribe audio")
            
            # Generate AI analysis
            analysis = self.generate_analysis(transcript)
            if not analysis:
                raise Exception("Failed to generate analysis")
            
            # Save report
            self.save_report(analysis, url)
            
            processing_time = time.time() - start_time
            
            return {
                'url': url,
                'success': True,
                'result': analysis,
                'processing_time': processing_time
            }
            
        except Exception as e:
            processing_time = time.time() - start_time
            self.logger.error(f"Failed to process {url}: {e}")
            return {
                'url': url,
                'success': False,
                'error': str(e),
                'processing_time': processing_time
            }
    
    def process_videos_parallel(self, video_urls):
        """Process videos in parallel for maximum performance"""
        results = []
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.optimal_settings['max_workers']) as executor:
            future_to_url = {
                executor.submit(self.process_single_video, url): url 
                for url in video_urls
            }
            
            for future in concurrent.futures.as_completed(future_to_url):
                url = future_to_url[future]
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    self.logger.error(f"Error processing {url}: {e}")
                    results.append({
                        'url': url,
                        'success': False,
                        'error': str(e),
                        'processing_time': 0
                    })
        
        return results
    
    def run_accelerated_pipeline(self):
        """Run the accelerated pipeline"""
        self.logger.info("Starting Self-Contained Nasdaq Trader Pipeline")
        
        # Optimize system
        self.optimize_system()
        
        # Load video URLs
        video_urls = self.load_video_urls()
        if not video_urls:
            self.logger.error("No video URLs found")
            return
        
        self.logger.info(f"Found {len(video_urls)} videos to process")
        
        # Process videos in parallel
        results = self.process_videos_parallel(video_urls)
        
        self.logger.info("Self-contained pipeline complete!")
        return results

# Initialize and run the self-contained trader
print("🚀 Starting SELF-CONTAINED Video Processing")
print("=" * 50)

# Initialize the self-contained trader
trader = SelfContainedNasdaqTrader()

# Run the pipeline
results = trader.run_accelerated_pipeline()

# Display results
if results:
    successful = sum(1 for r in results if r['success'])
    total_time = sum(r['processing_time'] for r in results)
    
    print(f"\n📊 Processing Results:")
    print(f"   Videos processed: {len(results)}")
    print(f"   Successful: {successful}")
    print(f"   Failed: {len(results) - successful}")
    print(f"   Total time: {total_time:.2f}s")
    print(f"   Average per video: {total_time/len(results):.2f}s")
    
    # Show individual results
    for result in results:
        if result['success']:
            print(f"   ✅ {result['url']} - {result['processing_time']:.2f}s")
        else:
            print(f"   ❌ {result['url']} - {result['error']}")
    
    print(f"\n📁 Generated files:")
    print(f"   - Reports: summary/ folder")
    print(f"   - Audio cache: video_cache/ folder")
    print(f"   - Transcripts: transcript_cache/ folder")
else:
    print("❌ No results generated")


🚀 Starting SELF-CONTAINED Video Processing


2025-10-12 08:30:07,767 | INFO | Starting Self-Contained Nasdaq Trader Pipeline
2025-10-12 08:30:07,767 | INFO | System optimized for high performance
2025-10-12 08:30:07,769 | INFO | Found 1 videos to process
2025-10-12 08:30:07,770 | INFO | Processing: https://www.youtube.com/watch?v=K8TFnwpDoAE
2025-10-12 08:30:07,773 | INFO | Using cached audio: video_cache\K8TFnwpDoAE_20251012.m4a
2025-10-12 08:30:07,774 | INFO | Using cached transcript for K8TFnwpDoAE


🚀 Self-Contained Nasdaq Trader Initialized
   System: 16 cores, 27.8GB RAM
   Optimal: 4 parallel videos


2025-10-12 08:31:20,638 | INFO | Self-contained pipeline complete!


Report saved: summary/report_K8TFnwpDoAE_20251012_083120.txt

📊 Processing Results:
   Videos processed: 1
   Successful: 1
   Failed: 0
   Total time: 72.87s
   Average per video: 72.87s
   ✅ https://www.youtube.com/watch?v=K8TFnwpDoAE - 72.87s

📁 Generated files:
   - Reports: summary/ folder
   - Audio cache: video_cache/ folder
   - Transcripts: transcript_cache/ folder


## 📊 Review Current Session Reports

**Review trading reports generated in this session**


In [3]:
# Review Current Session Reports
import json
import os
from pathlib import Path
from datetime import datetime

print("📊 Review Current Session Reports")
print("=" * 50)

# Check if we have results from the current session
if 'results' in locals() and results:
    print(f"📁 Found {len(results)} reports from current session")
    
    # Filter successful results only
    successful_results = [r for r in results if r['success']]
    
    if successful_results:
        print(f"\n📋 Current Session Reports ({len(successful_results)} successful):")
        print("-" * 60)
        
        for i, result in enumerate(successful_results, 1):
            url = result['url']
            processing_time = result['processing_time']
            
            # Extract video ID from URL
            video_id = url.split('v=')[-1].split('&')[0] if 'v=' in url else 'unknown'
            
            print(f"{i:2d}. Video ID: {video_id}")
            print(f"    🎬 URL: {url}")
            print(f"    ⏱️  Processing Time: {processing_time:.2f}s")
            print(f"    ✅ Status: Success")
            
            # Try to find the corresponding report file
            try:
                # Look for the most recent report for this video
                report_pattern = f"summary/report_{video_id}_*.txt"
                import glob
                report_files = glob.glob(report_pattern)
                
                if report_files:
                    # Get the most recent report
                    latest_report = max(report_files, key=os.path.getctime)
                    file_size = os.path.getsize(latest_report)
                    file_time = datetime.fromtimestamp(os.path.getctime(latest_report))
                    
                    print(f"    📄 Report: {os.path.basename(latest_report)}")
                    print(f"    📅 Generated: {file_time.strftime('%Y-%m-%d %H:%M:%S')}")
                    print(f"    📏 Size: {file_size:,} bytes")
                    
                    # Show first few lines of the report
                    try:
                        with open(latest_report, 'r', encoding='utf-8') as f:
                            lines = f.readlines()[:5]  # First 5 lines
                            print(f"    📋 Preview:")
                            for line in lines:
                                print(f"        {line.strip()}")
                            print(f"    ...")
                    except Exception as e:
                        print(f"    ❌ Error reading report: {e}")
                else:
                    print(f"    ⚠️  Report file not found for {video_id}")
            except Exception as e:
                print(f"    ❌ Error finding report: {e}")
            print()
        
        # Show the first successful report in detail
        if successful_results:
            first_result = successful_results[0]
            video_id = first_result['url'].split('v=')[-1].split('&')[0] if 'v=' in first_result['url'] else 'unknown'
            
            # Find the report file
            report_pattern = f"summary/report_{video_id}_*.txt"
            import glob
            report_files = glob.glob(report_pattern)
            
            if report_files:
                latest_report = max(report_files, key=os.path.getctime)
                print(f"📄 Viewing Latest Report: {os.path.basename(latest_report)}")
                print("=" * 60)
                
                try:
                    with open(latest_report, 'r', encoding='utf-8') as f:
                        content = f.read()
                        print(content)
                except Exception as e:
                    print(f"❌ Error reading report: {e}")
            else:
                print("⚠️  Report file not found")
    
    # Show failed results if any
    failed_results = [r for r in results if not r['success']]
    if failed_results:
        print(f"\n❌ Failed Results ({len(failed_results)}):")
        print("-" * 40)
        for i, result in enumerate(failed_results, 1):
            print(f"{i:2d}. {result['url']}")
            print(f"    ❌ Error: {result['error']}")
            print(f"    ⏱️  Time: {result['processing_time']:.2f}s")
            print()
    
    print("✅ Current session report review complete!")
    
else:
    print("⚠️  No results from current session found")
    print("💡 Run the processing pipeline first to generate reports")
    
    # Fallback: show all reports if no session results
    print("\n📁 Showing all reports in summary/ folder:")
    summary_dir = Path("summary")
    if summary_dir.exists():
        report_files = list(summary_dir.glob("*.txt"))
        if report_files:
            print(f"Found {len(report_files)} total reports")
            for i, report_file in enumerate(report_files, 1):
                print(f"{i:2d}. {report_file.name}")
        else:
            print("No reports found")
    else:
        print("Summary folder not found")


📊 Review Current Session Reports
📁 Found 1 reports from current session

📋 Current Session Reports (1 successful):
------------------------------------------------------------
 1. Video ID: K8TFnwpDoAE
    🎬 URL: https://www.youtube.com/watch?v=K8TFnwpDoAE
    ⏱️  Processing Time: 72.87s
    ✅ Status: Success
    📄 Report: report_K8TFnwpDoAE_20251012_083120.txt
    📅 Generated: 2025-10-12 08:31:20
    📏 Size: 31,564 bytes
    📋 Preview:
        Trading Analysis Report
        Video URL: https://www.youtube.com/watch?v=K8TFnwpDoAE
        Generated: 2025-10-12 08:31:20
        
    ...

📄 Viewing Latest Report: report_K8TFnwpDoAE_20251012_083120.txt
Trading Analysis Report
Video URL: https://www.youtube.com/watch?v=K8TFnwpDoAE
Generated: 2025-10-12 08:31:20

# TRADİNG ANALİZ RAPORU

## 📊 VİDEO BİLGİLERİ
- **Tarih**: 06 Haziran 2024
- **Kanal**: Videoda belirtilmemiş
- **Video Başlığı**: Videoda belirtilmemiş
- **Rapor Oluşturulma**: 06 Haziran 2024, 15:30 (Tahmini)

## 🎯 ÖZET
Piyasada g

## 🎯 Production Best Practices

### ✅ Quality Assurance
- **Source Validation**: All information comes from video transcripts only
- **Ticker Verification**: All ticker symbols are validated
- **Price Accuracy**: Only prices explicitly mentioned in videos
- **Anti-Hallucination**: No external data or assumptions

### 📋 Execution Checklist
- [ ] Verify `GEMINI_API_KEY` is set
- [ ] Add video URLs to `video_list.txt`
- [ ] Run processing pipeline
- [ ] Review generated reports
- [ ] Validate ticker symbols before trading
- [ ] Use proper risk management

### ⚠️ Important Disclaimers
- This analysis is based solely on video content
- All tickers and prices should be verified before trading
- Past performance does not guarantee future results
- Always use proper risk management

---

**🚀 Ready for professional trading analysis!**
